In [42]:
import pandas as pd
import numpy as np

turns = 'turns.csv'
turnsdf = pd.read_csv(turns, header=0, index_col='game_id')
turnsdf['move'] = turnsdf['move'].astype(str).replace('nan', 'NULL')

#get rid of all wildcards and intersections
wildcard = str.maketrans("abcdefghijklmnopqrstuvwxyz.", "0" * 27)

#scrabble points
scrabble_letter_values = {
    'A': 1, 'B': 3, 'C': 3, 'D': 2,
    'E': 1, 'F': 4, 'G': 2, 'H': 4,
    'I': 1, 'J': 8, 'K': 5, 'L': 1,
    'M': 3, 'N': 1, 'O': 1, 'P': 3,
    'Q': 10, 'R': 1, 'S': 1, 'T': 1,
    'U': 1, 'V': 4, 'W': 4, 'X': 8,
    'Y': 4, 'Z': 10, '0': 0
}


def points_above_expected(row):
    points_above = 0
    if row['turn_type'] == 'Play':
            move_word = [*row['move'].translate(wildcard)]
            move_expect = sum(scrabble_letter_values.get(letter, 0) for letter in move_word)
            points_above = row['points'] - move_expect
    return points_above

turnsdf['pae'] = turnsdf.apply(points_above_expected,axis=1)

print(turnsdf.head())



         turn_number   nickname     rack location      move  points  score  \
game_id                                                                      
1                  1  BetterBot  DDEGITT       8G       DIG      10     10   
1                  2      stevy  AEHOPUX       7H       HAP      18     18   
1                  3  BetterBot  DEELTTU       6I      LUTE      16     26   
1                  4      stevy  EMORSUX       5K        UM      16     34   
1                  5  BetterBot  ACDEITU       L5  ..DICATE      28     54   

        turn_type  pae  
game_id                 
1            Play    5  
1            Play   10  
1            Play   12  
1            Play   12  
1            Play   19  


In [43]:
total_pae = turnsdf.groupby(['game_id', 'nickname'])['pae'].agg(['sum', 'max', 'median']).reset_index()
print(total_pae)

        game_id   nickname  sum  max  median
0             1  BetterBot  230   61    13.0
1             1      stevy  346   89    20.0
2             2  BetterBot  307   73    14.0
3             2      Super  394   84    20.0
4             3  BetterBot  240   68    15.0
...         ...        ...  ...  ...     ...
145541    72771   HastyBot  303   83    13.0
145542    72772  BetterBot  334   70    18.0
145543    72772  Gtowngrad  307   60    15.5
145544    72773   HastyBot  240   36    20.0
145545    72773      adola  300   81    15.5

[145546 rows x 5 columns]


In [44]:
total_pae['count'] = total_pae.groupby('game_id').cumcount() + 1

pivot_df = total_pae.pivot_table(index='game_id', columns='count', values=['nickname', 'sum', 'max', 'median'], aggfunc='first', fill_value='')

pivot_df.columns = [f'{col[0]}{col[1]}' for col in pivot_df.columns]

pivot_df.reset_index(inplace=True)

# Display the result
print(pivot_df)

       game_id  max1  max2  median1  median2  nickname1       nickname2  sum1  \
0            1    61    89     13.0     20.0  BetterBot           stevy   230   
1            2    73    84     14.0     20.0  BetterBot           Super   307   
2            3    68    93     15.0     19.5  BetterBot       davidavid   240   
3            4    63    19     18.0      3.5  BetterBot  Inandoutworker   372   
4            5    73    42     16.0     13.5    STEEBot           stevy   340   
...        ...   ...   ...      ...      ...        ...             ...   ...   
72768    72769   182    73     22.0     12.5    STEEBot        browndav   521   
72769    72770    96    17     19.5      4.0   HastyBot       samsiah06   461   
72770    72771    73    83     13.0     13.0       BB-8        HastyBot   281   
72771    72772    70    60     18.0     15.5  BetterBot       Gtowngrad   334   
72772    72773    36    81     20.0     15.5   HastyBot           adola   240   

       sum2  
0       346  